# Finding Correlations


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from typing import Tuple
import plotly.express as px
import plotly.graph_objects as go
import altair as alt
from IPython.display import display
from plotly.graph_objects import Figure

# 😉 Go Blue!
maize = "#FFCB05"  # also rgb(255, 203, 5)
blue = "#00274C"  # also rgb(0, 39, 76)

df = pd.read_csv("./data/complete_dataset_with_interpolation.csv")


In [2]:
def build_correlation_df(
    df: pd.DataFrame, years: int, x: str, y: str
) -> Tuple[pd.DataFrame, float, float]:
    df_corr = df[df["indicator"].eq(x) | (df["indicator"].eq(y))]

    # Convert values to x and y
    df_corr = df_corr.replace({x: "x", y: "y"})

    # Limit data to the past n `years`
    df_corr = df_corr[["country", "indicator", *map(str, range((2020 - years), 2020))]]

    df_corr = (
        df_corr.melt(id_vars=["country", "indicator"])
        .groupby(["country", "indicator"])
        .mean(numeric_only=True)
        .reset_index()
        .pivot(index="country", columns="indicator")
        .droplevel(0, axis="columns")
        .reset_index()
        .dropna()
    )

    p_corr, p_pval = stats.pearsonr(df_corr["x"], df_corr["y"])
    s_corr, s_pval = stats.spearmanr(df_corr["x"], df_corr["y"])

    return df_corr, p_corr, p_pval, s_corr, s_pval


Remove the two aggregate sources in our data, the EU and World totals.


In [3]:
find_corr_df = df[~df["c_code"].isin(["EUU", "WLD"])]


Generate list of sectors and gas types to try in our correlation function.


In [4]:
sectors = find_corr_df[find_corr_df["gas"].notna()]["indicator"].unique()

sectors


array(['Agriculture', 'Building', 'Bunker Fuels', 'Electricity/Heat',
       'Energy', 'Fugitive Emissions', 'Industrial Processes',
       'Land-Use Change and Forestry', 'Manufacturing/Construction',
       'Other Fuel Combustion', 'Total excluding LUCF',
       'Total including LUCF', 'Transportation', 'Waste'], dtype=object)

In [5]:
gases = find_corr_df[find_corr_df["gas"].notna()]["gas"].unique()

gases


array(['All GHG', 'CH4', 'N2O', 'CO2', 'F-Gas'], dtype=object)

Generate a list of all correlations for these sectors.


In [6]:
years = 30
corr_sectors = ["Land-Use Change and Forestry", "Total excluding LUCF", "Total including LUCF"]
indicators = df[~df["indicator"].isin(sectors)]["indicator"].unique()

results = []
for x in indicators:
    for y in corr_sectors:
        try:
            out_df, p_r, p_p, s_r, s_p = build_correlation_df(find_corr_df, years, x, y)
            results.append((x, y, len(out_df), p_r, p_p, s_r, s_p))
        except:
            # The correlation doesn't work if there are singular data points or if the indicators are identical.
            continue

# Sort by Pearson's r values
results = sorted(results, key=lambda x: x[3], reverse=True)


In [7]:
correlation_df = pd.DataFrame(
    results,
    columns=[
        "indicator",
        "ghg_sector",
        "n",
        "pearson_r",
        "pearson_p_value",
        "spearman_r",
        "spearman_p_value",
    ],
)

# Generate complete correlation output to analyze indicators
# correlation_df.to_csv("./data/correlations.csv", index=False)


In [8]:
correlation_df.head(10)


,indicator,ghg_sector,n,pearson_r,pearson_p_value,spearman_r,spearman_p_value
0,Proportion of population living in multidimens...,Land-Use Change and Forestry,2,1.000000,1.000000e+00,1.000000,NaN
1,Proportion of population living in multidimens...,Total excluding LUCF,2,1.000000,1.000000e+00,1.000000,NaN
2,Proportion of population living in multidimens...,Total including LUCF,2,1.000000,1.000000e+00,1.000000,NaN
3,Average share of weighted deprivations of tota...,Land-Use Change and Forestry,3,0.990559,8.754984e-02,1.000000,0.000000e+00
4,Proportion of households living in multidimens...,Land-Use Change and Forestry,3,0.990559,8.754984e-02,1.000000,0.000000e+00
5,"GDP, PPP (current international $)",Total excluding LUCF,186,0.944926,3.461746e-91,0.954670,9.019016e-99
6,"GDP, PPP (current international $)",Total including LUCF,186,0.944847,3.939432e-91,0.904849,3.740844e-70
7,"GDP, PPP (constant 2017 international $)",Total excluding LUCF,182,0.922108,3.828819e-76,0.953408,1.297847e-95
8,"GDP, PPP (constant 2017 international $)",Total including LUCF,182,0.920276,2.853189e-75,0.902026,1.404674e-67
9,Average share of weighted deprivations of tota...,Land-Use Change and Forestry,3,0.854712,3.474671e-01,1.000000,0.000000e+00


In [9]:
correlation_df.tail(10)


,indicator,ghg_sector,n,pearson_r,pearson_p_value,spearman_r,spearman_p_value
539,Proportion of children living in child-specifi...,Total including LUCF,9,-0.578029,0.103051,-0.200000,0.605901
540,Average proportion of deprivations for people ...,Total including LUCF,21,-0.598074,0.004187,0.027273,0.906584
541,Proportion of children living in child-specifi...,Total including LUCF,11,-0.618452,0.042529,-0.290909,0.385457
542,Average proportion of deprivations for people ...,Total including LUCF,21,-0.637307,0.001887,-0.103896,0.654027
543,Proportion of children living in child-specifi...,Total excluding LUCF,11,-0.649502,0.030556,-0.290909,0.385457
544,Average proportion of deprivations for people ...,Total excluding LUCF,21,-0.658548,0.001170,-0.177922,0.440350
545,Proportion of children living in child-specifi...,Total excluding LUCF,9,-0.658939,0.053573,-0.133333,0.732368
546,Average proportion of deprivations for people ...,Total excluding LUCF,21,-0.693483,0.000490,-0.294805,0.194527
547,Average share of weighted deprivations of tota...,Total excluding LUCF,3,-0.897972,0.290081,-0.500000,0.666667
548,Proportion of households living in multidimens...,Total excluding LUCF,3,-0.897972,0.290081,-0.500000,0.666667


In [10]:
def build_correlation_plot(
    df: pd.DataFrame, x: str, y: str, corr: float, pval: float, log_scale=False
) -> Figure:
    df_plt = (df[df["x"].gt(0) & df["y"].gt(0)]) if log_scale else df
    x_label = f"{x} (Log Scaled)" if log_scale else x
    y_label = f"{y} (Log Scaled)" if log_scale else y

    fig = px.scatter(
        df_plt,
        x="x",
        y="y",
        log_x=(True if log_scale else False),
        log_y=(True if log_scale else False),
        labels={"x": x_label, "y": y_label},
        title=f"Correlation between {x}<br>and {y}<br>Pearson's r: {corr}, P-value: {pval}, n={len(df_plt)}<br>",
        hover_data=["country"],
        color_discrete_sequence=[blue],
        trendline="ols",
        trendline_options=({"log_x": True, "log_y": True} if log_scale else None),
        trendline_color_override=maize,
    )

    fig.update_layout(width=800, height=600)

    return fig


In [11]:
years = 30
x = "GDP, PPP (current international $)"
y = "Total including LUCF"

df_xy, p_corr, p_pval, s_corr, s_pval = build_correlation_df(find_corr_df, years, x, y)
build_correlation_plot(df_xy, x, y, p_corr, p_pval, log_scale=True).show()


Recreate the plot above without log scaling to use as an inset.


In [12]:
build_correlation_plot(df_xy, x, y, p_corr, p_pval).update_layout(
    width=600, height=600
).show()


We also created correlation plots for various poverty indicators and Land Use Change and Forestry (LUCF), but these ultimately didn't make it into the report because of their low Pearson r values and relevance to our data narrative.


In [13]:
years = 30
x = "Proportion of population below international poverty line (%)PERCENTALLAGEALLAREABOTHSEX"
y = "Total including LUCF"

df_xy, p_corr, p_pval, s_corr, s_pval = build_correlation_df(find_corr_df, years, x, y)
build_correlation_plot(df_xy, x, y, p_corr, p_pval, log_scale=True).show()


This multidimensional poverty indicator shows a promising negative correlation, but the low sample size (20) and scatterplot prove that there really isn't much to go on here other than pointing out Mexico as an outlier in this set.


In [14]:
years = 30
x = "Average proportion of deprivations for people multidimensionally poor (%)PERCENTALLAGEURBANBOTHSEX"
y = "Total including LUCF"

df_xy, p_corr, p_pval, s_corr, s_pval = build_correlation_df(find_corr_df, years, x, y)
build_correlation_plot(df_xy, x, y, p_corr, p_pval, log_scale=True).show()


In [15]:
years = 30
x = "Proportion of population living in multidimensional poverty (%)PERCENTALLAGEALLAREABOTHSEX"
y = "Total including LUCF"

df_xy, p_corr, p_pval, s_corr, s_pval = build_correlation_df(find_corr_df, years, x, y)
build_correlation_plot(df_xy, x, y, p_corr, p_pval, log_scale=True).show()


We also looked for interesting correlations between GDP, Poverty, and LUCF which represents emissions or removals of greenhouse gases from the result of human manipulation of land and forestry. This can mean large carbon sinks for countries with swaths of forests (especially rainforests), but also carbon sources where deforestation has occurred.

Unfortunately, with weak correlations, we were unable to add any interesting insights to our analysis.


In [16]:
years = 30
x = "GDP, PPP (current international $)"
y = "Land-Use Change and Forestry"

df_xy, p_corr, p_pval, s_corr, s_pval = build_correlation_df(find_corr_df, years, x, y)
build_correlation_plot(df_xy, x, y, p_corr, p_pval).show()


In [17]:
years = 30
x = "Proportion of population below international poverty line (%)PERCENTALLAGEALLAREABOTHSEX"
y = "Land-Use Change and Forestry"

df_xy, p_corr, p_pval, s_corr, s_pval = build_correlation_df(find_corr_df, years, x, y)
build_correlation_plot(df_xy, x, y, p_corr, p_pval).show()


In [18]:
%reload_ext watermark

%watermark -iv -v -m

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

altair: 4.2.0
plotly: 5.10.0
numpy : 1.23.3
scipy : 1.9.1
sys   : 3.10.6 (main, Aug 30 2022, 05:12:36) [Clang 13.1.6 (clang-1316.0.21.2.5)]
pandas: 1.5.0

